In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import matplotlib.pyplot as plt
from random import randint
import pickle

In [3]:
# this list holds all the image filename
filepath = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filepath.append(os.path.join(dirname, filename))
       

In [5]:
img = load_img(filepath[0], target_size=(224,224))
# convert from 'PIL.Image.Image' to numpy array
img = np.array(img)

print(img.shape)

In [6]:
reshaped_img = img.reshape(1,224,224,3)
print(reshaped_img.shape)

In [7]:
x = preprocess_input(reshaped_img)

In [8]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array(for ease of processing the data)
    img = load_img(file, target_size=(224,224))

    img = np.array(img) 

    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [9]:
data = {}

for tshirt in filepath:
    
    feat = extract_features(tshirt,model)
    data[tshirt] = feat

        
          
 

 
filenames = np.array(list(data.keys()))

feat = np.array(list(data.values()))


In [20]:
data

In [10]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

In [25]:
from sklearn.cluster import AffinityPropagation
affprop = AffinityPropagation(affinity="euclidean", damping=0.5).fit(np.array(x))

In [26]:
affprop.labels_

In [30]:
cluster_map_a = pd.DataFrame()
cluster_map_a['data_index'] = filepath
cluster_map_a['cluster'] = affprop.labels_

In [31]:
cluster_map_a

In [22]:
kmeans = KMeans(n_clusters=300, random_state=22)
kmeans.fit(x)

In [28]:
cluster_map_k = pd.DataFrame()
cluster_map_k['data_index'] = filepath
cluster_map_k['cluster'] = kmeans.labels_

In [63]:
def view_cluster(file, filedf):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    i = filedf.data_index[filedf.data_index == file].index.tolist()
    filecluster = filedf.cluster[i[0]]
    files= []
    for j in range (filedf.shape[0]):
        if filedf.cluster[j] == filecluster:
            files.append(filedf.data_index[j])
    
    if len(files) > 10:
        files = files[:10]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

    print(files)

In [46]:
def viewcluster(file, filedf):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    h = filedf.data_index[filedf.data_index == file].index.tolist()
    print(h)

In [64]:
view_cluster('/kaggle/input/flixstock/bottoms_resized_png/13079565VVD.png', cluster_map_a)